## Este código limpia y agrega los datos de las tendencias de Google

Las busquedas de Airbnb que se ocuparán son las siguientes:
1. "airbnb ciudad de mexico"
2. "airbnb cdmx"
3. "airbnb mexico city"

"airbnb mexico city" es utilizado para captar la demanda extranjera de Airbnb en la Ciudad de México.

### Importacion de librerias y carga de las bases de datos

En este punto juntaré todos los datos de búsquedas de airbnb con distintos filtros y a distintos niveles en un solo dataframe. Esto es porque será más fácil hacerlo de esta manera al tratar de juntarlos a la base completa. La modificación de las bases la hago casi por completo dentro del mismo loop. 

In [1]:
import pandas as pd
import os
import re

In [2]:
#Definimos la carpeta en donde están los datos
carpeta = r"C:\Users\santo\Documents\CIDE\Tesis\datos_google"

In [3]:
#Listar los archivos en la carpeta que ocuparé
archivos = [base for base in os.listdir(carpeta) if base.startswith("gtrends") and base.endswith(".csv")]

#Lista para guardar todos los dataframe procesados
dataframes = []

#Creo un loop para cargar el archivo y hacer las modificaciones a las variables desde el inicio
for archivo in archivos:
    ruta_completa = os.path.join(carpeta, archivo)
    
    #Extraer datos del nombre de las bases para la creación de las variables en el dataframe unido
    match = re.search(r"gtrends_(global|nac)_(\d{4}_\d{4})", archivo) #Busca una coincidencia en el nombre del archivo que cumpla con el patron que escribimos. \d{4} le está diciendo que busque 4 dígitos 
    if not match:
        continue #si el archivo no cumple con el patron, lo brinca
    nivel = match.group(1)  #Obtiene lo que estaba en el primer paréntesis del patrón: "global" o "nac"
    periodo = match.group(2) #Obtiene lo que estaba en el segundo paréntesis del patrón
    
    #Se carga el archivo y se especifíca que los encabezados están en la segunda fila
    df = pd.read_csv(ruta_completa, header = 1)
    
    #Detectamos si los datos están en temporalidad semanal o mensual
    if 'Semana' in df.columns:
        #Si es en semana, entonces convertimos la columna a formato de a fecha y creamos una nueva columna mes a partir de esa fecha
        df['Semana'] = pd.to_datetime(df['Semana'])
        df['Mes'] = df['Semana'].dt.to_period('M')
        df_mensual = df.groupby('Mes').mean().reset_index() #Agrupar por mes para poder tomar el valor promedio que hubo en cada columna en cada mes. El reset_index() lo ocupo porque al utilizar groupby, pandas convierte la columna por la que agrupe (en este caso 'Mes') en el nuevo índice del dataframe. Al utilizar reset_index lo que hago es volverlo nuevamente una columna normal.
    else: 
        #Si ya está mensual, renombramos la columna por seguridad y la convertimos a fecha
        df.rename(columns = {df.columns[0]: 'Mes'}, inplace = True)
        #Convertir columna de fechas a formato de fecha y generar una columna de mes
        df['Mes'] = pd.to_datetime(df['Mes']).dt.to_period('M')
        df_mensual = df.copy()
        
    #Renombrar columnas según la búsqueda, su nivel, y su periodo. Esto es para poder identificarlas más fácil en la base de datos final
    columnas_nuevas = {}
    for col in df_mensual.columns:
        if col == 'Mes':
            continue
        if 'airbnb cdmx' in col.lower(): #ignoramos mayúsculas con col.lower() y no utilizamos == porque no buscamos una coincidencia exacta sino si cierto texto está contenido dentro del nombre de la columna 
            columnas_nuevas[col] = f"a_cdmx_{nivel}_{periodo}"
        elif 'airbnb ciudad de mexico' in col.lower():
            columnas_nuevas[col] = f"a_ciumex_{nivel}_{periodo}"
        elif 'airbnb mexico city' in col.lower():
            columnas_nuevas[col] = f"a_mexcity_{nivel}_{periodo}"
    df_mensual.rename(columns = columnas_nuevas, inplace = True)
    
    #Por seguridad, para que no tengamos otra columna que no necesitamos, conservamos solo la columna de 'Mes' y las otras que acaban de ser renombreadas
    columnas_finales = ['Mes'] + list(columnas_nuevas.values())
    df_mensual = df_mensual[columnas_finales]
    
    #Agregar el dataframe modificado a la lista vacia 
    dataframes.append(df_mensual)


## Unión en un dataframe completo

In [4]:
#Unir todos los dataframes por la columna 'Mes'
if dataframes: #verifica que la lista tenga algo
    df_completo = dataframes[0] # Empieza con el primer dataframe de la lista como base
    for df in dataframes[1:]: #continua con los dataframes restantes uno por uno
        df_completo = pd.merge(df_completo, df, on = 'Mes', how = 'outer')
else:
    df_completo = pd.DataFrame() #Por si no cargó nada
    
#Ordenar por fecha el dataframe
df_completo.sort_values('Mes', inplace=True)

In [5]:
df_completo

,Mes,a_cdmx_global_2019_2020,a_mexcity_global_2019_2020,a_ciumex_global_2019_2020,a_cdmx_global_2019_2021,a_ciumex_global_2019_2021,a_mexcity_global_2019_2021,a_cdmx_global_2019_2025,a_ciumex_global_2019_2025,a_mexcity_global_2019_2025,...,a_ciumex_nac_2019_2020,a_cdmx_nac_2019_2021,a_ciumex_nac_2019_2021,a_mexcity_nac_2019_2021,a_cdmx_nac_2019_2025,a_ciumex_nac_2019_2025,a_mexcity_nac_2019_2025,a_cdmx_nac_2023_2025,a_ciumex_nac_2023_2025,a_mexcity_nac_2023_2025
0,2018-12,44.00,78.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-01,37.00,64.75,20.75,37.0,17.00,92.00,NaN,NaN,NaN,...,10.0,33.00,0.0,19.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2019-02,31.75,67.00,19.75,37.0,22.50,74.25,12.0,7.0,25.0,...,10.0,35.75,10.0,0.0,25.0,9.0,8.0,NaN,NaN,NaN
3,2019-03,40.20,71.40,26.00,47.6,22.40,72.00,16.0,10.0,24.0,...,19.8,43.00,13.6,3.2,29.0,14.0,9.0,NaN,NaN,NaN
4,2019-04,45.50,54.75,25.75,49.0,27.25,62.00,16.0,8.0,21.0,...,12.0,47.75,0.0,5.5,32.0,12.0,9.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,2024-09,NaN,NaN,NaN,NaN,NaN,NaN,45.0,9.0,24.0,...,NaN,NaN,NaN,NaN,82.0,13.0,8.0,74.20,10.0,2.6
70,2024-10,NaN,NaN,NaN,NaN,NaN,NaN,49.0,8.0,29.0,...,NaN,NaN,NaN,NaN,94.0,7.0,9.0,71.50,0.0,5.0
71,2024-11,NaN,NaN,NaN,NaN,NaN,NaN,48.0,10.0,31.0,...,NaN,NaN,NaN,NaN,86.0,13.0,11.0,72.25,8.5,7.5
72,2024-12,NaN,NaN,NaN,NaN,NaN,NaN,42.0,7.0,35.0,...,NaN,NaN,NaN,NaN,81.0,9.0,9.0,70.60,5.0,2.2


## Exportar la base completa

In [6]:
carpeta_final = r"C:\Users\santo\Documents\CIDE\Tesis\datos_unidos"

df_completo.to_csv(os.path.join(carpeta_final, "gtrends_limpio.csv"), index = False, encoding='utf-8-sig')